## Example of AH3 molecular
### Outline
In this example, we use the codebase to solve the orbital interaction in AH3 molecular. It's molecular structure is given by `automaticTB.examples.structures.AH3` where a carbon atom is used as A with $2s$ and $2p$ orbitals. 

The result is given in Section 4.8 of Albright's book *Orbital Interactions in Chemistry*. To arrive at the result, we first obtain all the symmetry adapted linear combination:

### Finding the symmetry adopted linear combination (SALCs)
Since we have in total 7 starting atomic orbital, we arrive at 7 linear combination in total with representation: $2\times A'$, $1\times A''$, $2\times E'$, furthermore, each subspace of the $E'$ representation are separated into symmetric and anti-symmetric components $A$ and $B$. Interaction is only allowed for linear combinations having the same symmetric properties. 

In [4]:
from automaticTB.structure import LocalSite, CrystalSite, CenteredEquivalentCluster
from automaticTB.structure.sitesymmetry import get_point_group_as_SiteSymmetryGroup
import numpy as np

group = get_point_group_as_SiteSymmetryGroup("-6m2")
w = np.sqrt(3) / 2

center_crystalsite = CrystalSite(
        site = LocalSite(6, np.array([0.0, 0.0, 0.0]) * 2.0),
        absolute_position = np.array([0.0, 0.0, 0.0]) * 2.0,
        index_pcell = 0, 
        equivalent_index = 0, 
        translation = np.zeros(3),
        orbitals = "2s2p"
    )
other_crystalsites = [
        CrystalSite(
            site = LocalSite(1, np.array([0.0, 1.0, 0.0]) * 2.0),
            absolute_position = np.array([0.0, 1.0, 0.0]) * 2.0,
            index_pcell = 1, 
            equivalent_index = 1,
            translation = np.zeros(3),
            orbitals = "1s"
        ),
        CrystalSite(
            site = LocalSite(1, np.array([  w,-0.5, 0.0]) * 2.0),
            absolute_position = np.array([  w,-0.5, 0.0]) * 2.0,
            index_pcell = 2, 
            equivalent_index = 1,
            translation = np.zeros(3),
            orbitals = "1s"
        ),
        CrystalSite(
            site = LocalSite(1, np.array([-1 * w,-0.5, 0.0]) * 2.0),
            absolute_position = np.array([-1 * w,-0.5, 0.0]) * 2.0,
            index_pcell = 3, 
            equivalent_index = 1,
            translation = np.zeros(3),
            orbitals = "1s"
        ),
]

ah3 = CenteredEquivalentCluster(
    center_site=center_crystalsite,
    neighbor_sites=other_crystalsites,
    sitesymmetrygroup=group
)

In [5]:
from automaticTB.interaction import get_InteractingAOSubspaces_from_cluster

interacting_ao_subspace = get_InteractingAOSubspaces_from_cluster(ah3)
for subspace in interacting_ao_subspace:
    print(subspace)

H_1s_linear_combination = interacting_ao_subspace[0].r_subspace
for nlc in H_1s_linear_combination.namedlcs:
    print(nlc)

-------------------------------------------------------
Block: ( C 2s) -- r =   2.0A --> (H  1s)
left                          right                         
A_'_1 @ 1^th A_'_1            A_'_1 @ 1^th A_'_1            
                              E_'->A_1 @ 1^th E_'           
                              E_'->B_2 @ 1^th E_'           
-------------------------------------------------------
-------------------------------------------------------
Block: ( C 2p) -- r =   2.0A --> (H  1s)
left                            right                           
A_''_2 @ 1^th A_''_2            A_'_1 @ 1^th A_'_1              
E_'->A_1 @ 1^th E_'             E_'->A_1 @ 1^th E_'             
E_'->B_2 @ 1^th E_'             E_'->B_2 @ 1^th E_'             
-------------------------------------------------------
A_'_1 @ 1^th A_'_1
├─  H @ (  0.00,  2.00,  0.00) :: +0.577(1-s)
├─  H @ (  1.73, -1.00,  0.00) :: +0.577(1-s)
└─  H @ ( -1.73, -1.00,  0.00) :: +0.577(1-s)
E_'->A_1 @ 1^th E_'
├─  H @ (  0.

### Plotting the linear combination
To plot the symmetry adopted wavefunction, use the following function

In [6]:
from automaticTB.functions import plot_molecular_wavefunction_from_linear_combination

print("Plotting Symmetry adopted Linear Combinations")
functions_to_plot = [
    "E_'->A_1 @ 1^th E_'",  # oriented p orbitals
]
for nlc in H_1s_linear_combination.namedlcs:
    if str(nlc.name) not in functions_to_plot: continue
    filename = f"{str(nlc.name)}.cube"
    plot_molecular_wavefunction_from_linear_combination(
        nlc.lc, filename
    )
    print(f"wavefunction plotted to {filename}")

Plotting Symmetry adopted Linear Combinations
wavefunction plotted to E_'->A_1 @ 1^th E_'.cube


### Finding the free interaction
Between the 7 linear combinations, we have in total $7\times 7$ interactions between the states. However, only 5 interaction is allowed. This offers a simplification if to understand the chemical interaction and solving of the energy eigenstates. 

In [7]:
from automaticTB.interaction import CombinedAOSubspaceInteraction

combined_interaction = CombinedAOSubspaceInteraction(interacting_ao_subspace)
for pair in combined_interaction.free_AOpairs:
    print(pair)

 > Pair:  C-00      2s ->  H-03      1s r = ( -1.73, -1.00,  0.00)
 > Pair:  C-00     2px ->  H-03      1s r = ( -1.73, -1.00,  0.00)
